In [188]:
import pandas as pd
import numpy as np
        

In [7]:
df = pd.read_csv('Data/BHCF20250331.txt', delimiter = '^')


/var/folders/17/1lp6v8rx7hg46v35nrbf4mk00000gn/T/ipykernel_7843/2340049902.py:1: DtypeWarning: Columns (2088,2089,2092,2097,2100,2101,2141,2152,2153,2154,2157,2158,2167,2185,2201,2202,2203,2204,2205,2206,2207,2208,2216,2219,2220,2221,2222) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/BHCF20250331.txt', delimiter = '^')


In [13]:
dfq2 = pd.read_csv('Data/BHCF20250630 3.txt', delimiter = '^')

/var/folders/17/1lp6v8rx7hg46v35nrbf4mk00000gn/T/ipykernel_7843/3343375304.py:1: DtypeWarning: Columns (2088,2089,2091,2092,2094,2095,2098,2099,2100,2101,2132,2141,2151,2152,2153,2154,2155,2156,2157,2158,2160,2161,2162,2163,2165,2166,2167,2171,2173,2174,2177,2178,2179,2181,2183,2184,2185,2200,2201,2202,2203,2204,2205,2206,2207,2208,2215,2216,2217,2219,2220,2221,2222) have mixed types. Specify dtype option on import or set low_memory=False.
  dfq2 = pd.read_csv('Data/BHCF20250630 3.txt', delimiter = '^')


In [8]:
print(df.columns.str[:4].unique())

Index(['RSSD', 'BHBC', 'BHCA', 'BHCB', 'BHCK', 'BHCM', 'BHCP', 'BHCT', 'BHCW',
       'BHCX', 'BHCY', 'BHDM', 'BHFN', 'BHOD', 'BHPA', 'BHPX', 'BHSP', 'BHSX',
       'BHTX', 'TEXT'],
      dtype='object')


In [9]:
dddf = pd.read_csv('/Users/jacksonlipfert/Downloads/MDRM/MDRM_CSV.csv')

In [10]:
#dddf.head()
short_dddf = dddf[['Mnemonic','Item Code','Item Name']]


short_dddf['Full Code'] = short_dddf['Mnemonic'] + short_dddf['Item Code']
important_codes = ['RSSD', 'BHBC', 'BHCA', 'BHCB', 'BHCK', 'BHCM', 'BHCP', 'BHCT', 'BHCW',
       'BHCX', 'BHCY', 'BHDM', 'BHFN', 'BHOD', 'BHPA', 'BHPX', 'BHSP', 'BHSX',
       'BHTX', 'TEXT']
short_dddf = short_dddf[short_dddf['Mnemonic'].isin(important_codes)]


/var/folders/17/1lp6v8rx7hg46v35nrbf4mk00000gn/T/ipykernel_7843/756459364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_dddf['Full Code'] = short_dddf['Mnemonic'] + short_dddf['Item Code']


In [159]:
# create mapper


column_mapper = {}

#Columns that appear across multiple types of companies with the same balance sheet name, need to keep code to disambiguate
special_columns = [
    'TOTAL ASSETS',
    'TOTAL EQUITY',
    'TOTAL EQUITY CAPITAL',
    'NET INCOME',
    'NET INCOME (LOSS)'
    ]

def get_name_from_full_code(fullcode, df):
    '''get column name from full code found in data dictionary'''
    column = df['Item Name'][df['Full Code'] == fullcode]
    col_name = column.iloc[0]
    return col_name


#Loop over short data dictionary and populate column mapper with name
for code in short_dddf['Full Code']:
    column_mapper[code] = get_name_from_full_code(code,short_dddf)


def disambiguate_column_mapper(column_mapper, special_columns):
    '''Loop over column mapper and replace ambigious columns (found in special_columns)'''

    for key,value in column_mapper.items():
        if value in special_columns:
            column_mapper[key] = f'{value}: {key}' 
    return column_mapper

#Final column mapper
disambig_column_mapper = disambiguate_column_mapper(column_mapper, special_columns)

#check to make sure columns have been renamed
column_name_in_question = 'NET INCOME'
for key,value in disambig_column_mapper.items():
    if column_name_in_question in str(value):
        print(value)
    


    

NET INCOME (LOSS): BHBC4340
NET IMPACT ON NET INCOME (BEFORE TAXES) OF THE SALE OF OTHER ASSETS WITH RECOURSE
NET IMPACT ON NET INCOME (BEFORE TAXES) OF LOAN ASSETS THAT WERE SOLD WITH RECOURSE
NET IMPACT ON NET INCOME (BEFORE TAXES) OF DEFEASED DEBT
NET INCOME (LOSS): BHCK4340
NET INCOME BEFORE INCOME TAXES; EXTRAORDINARY ITEMS; AND OTHER ADJUSTMENTS ON A FULLY TAXABLE EQUIVALENT BASIS
NET INCOME FOR ALL INSURANCE-RELATED ACTIVITIES
NET INCOME: BHCKC246
NET INCOME: BHCKC250
LESS: NET INCOME (LOSS) ATTRIBUTABLE TO NONCONTROLLING (MINORITY) INTERESTS
NET INCOME (LOSS) ATTRIBUTABLE TO BANK AND NONCONTROLLING MINORITY INTERESTS
NET INCOME FROM SERVICE CORPORATION/SUBSIDIARIES (SEE NOTE BELOW)
CASH FLOWS FROM OPERATING ACTIVITIES: ADJUSTMENTS TO RECONCILE NET INCOME TO NET CASH PROVIDED BY OPERATING ACTIVITIES - PROVISION FOR DEFERRED INCOME TAXES
CASH FLOWS FROM OPERATING ACTIVITIES: ADJUSTMENTS TO RECONCILE NET INCOME TO NET CASH PROVIDED BY OPERATING ACTIVITIES - (GAIN) OR LOSS ON SALES

In [161]:
renamed_df = df.rename(columns=disambig_column_mapper)
renamed_dfq2 = dfq2.rename(columns=disambig_column_mapper)


In [156]:
#Column Finder

#print(renamed_dfq2.columns.str.contains('NAME'))

search_string = 'NET INCOME'
matching_columns = renamed_dfq2.columns[renamed_dfq2.columns.str.contains(search_string)]
print(matching_columns)
#print(renamed_dfq2['REPORTING DATE (CC;YR;MO;DA)'])
# i = 0
# for name in renamed_dfq2['ENTITY SHORT NAME']:
#     i += 1
#     if i > 50:
#         break
#     else:
#         print(name)
#target_bank = 'FIRST NAT OF NE'
#print(renamed_dfq2[renamed_dfq2["ENTITY SHORT NAME"] == target_bank])
#print(renamed_df['AVERAGE TOTAL ASSETS (NET OF DEDUCTIONS)'].loc[2])
#print(renamed_dfq2['AVERAGE TOTAL ASSETS (NET OF DEDUCTIONS)'].loc[5])
#print(renamed_df["ENTITY SHORT NAME"].loc[2])


Index(['NET INCOME (LOSS)', 'NET INCOME (LOSS)',
       'NET INCOME BEFORE INCOME TAXES; EXTRAORDINARY ITEMS; AND OTHER ADJUSTMENTS ON A FULLY TAXABLE EQUIVALENT BASIS',
       'NET INCOME', 'NET INCOME',
       'LESS: NET INCOME (LOSS) ATTRIBUTABLE TO NONCONTROLLING (MINORITY) INTERESTS',
       'NET INCOME (LOSS) ATTRIBUTABLE TO BANK AND NONCONTROLLING MINORITY INTERESTS',
       'CASH FLOWS FROM OPERATING ACTIVITIES: ADJUSTMENTS TO RECONCILE NET INCOME TO NET CASH PROVIDED BY OPERATING ACTIVITIES - PROVISION FOR DEFERRED INCOME TAXES',
       'CASH FLOWS FROM OPERATING ACTIVITIES: ADJUSTMENTS TO RECONCILE NET INCOME TO NET CASH PROVIDED BY OPERATING ACTIVITIES - (GAIN) OR LOSS ON SALES OF ASSETS',
       'CASH FLOWS FROM OPERATING ACTIVITIES: ADJUSTMENTS TO RECONCILE NET INCOME TO NET CASH PROVIDED BY OPERATING ACTIVITIES - EQUITY IN UNDISTRIBUTED (EARNINGS) LOSSES OF SUBSIDIARIES',
       'CASH FLOWS FROM OPERATING ACTIVITIES: ADJUSTMENTS TO RECONCILE NET INCOME TO NET CASH PROVIDE

Ok, so what needs to happen now? I have two dataframes both of which have the same columns and I want to mix them together in such a way that I can compare values across time

I have a reporting date which matches the file date which is good. I could union the data and use the bank name and reporting date to compare over time?

In [172]:
# Attempt to union
union_df = pd.concat([renamed_df, renamed_dfq2], ignore_index=True)



In [196]:
#print(union_df[union_df['ENTITY SHORT NAME'] == target_bank])
filtered_df = union_df[['ENTITY SHORT NAME','TOTAL EQUITY CAPITAL: BHCK3210','NET INCOME (LOSS): BHBC4340','REPORTING DATE (CC;YR;MO;DA)']].copy()
#filtered_df = filtered_df[filtered_df['TOTAL ASSETS'] > 0]
#print(filtered_df.head())
grouped_df = filtered_df.groupby(['ENTITY SHORT NAME']).agg(
    assets = ('TOTAL EQUITY CAPITAL: BHCK3210',"mean"),
    dates = ('REPORTING DATE (CC;YR;MO;DA)' , 'count'),
    net_income = ('NET INCOME (LOSS): BHBC4340', 'mean')
)
net_income_df = grouped_df[(np.isnan(grouped_df['net_income']) == False) & (grouped_df['net_income'] != 0)]
print(net_income_df.head(20))

                        assets  dates  net_income
ENTITY SHORT NAME                                
ACNB CORP             391017.0      2      -141.5
CAPITAL ONE FC      87248851.5      2   1517926.0
CHOICEONE FS          429414.5      2       448.0
CONNECTONE BC        1374685.0      2      4542.0
EVERBANK FNCL CORP   3704783.0      2    -37161.0
FIRST BUSEY CORP     2296076.0      2     16285.0
HOPE BC              2192075.0      2     -9020.0
NBT BC               1685470.5      2     -4546.0
OLD NAT BC           7330520.5      2    -33795.0
UNITED BSHRS         5339495.0      4     -9785.0
WESBANCO             3800399.5      2    -33155.0


In [203]:
#Calculate volitility of earnings

VOE_bank = 'WESBANCO'
VOE_df = union_df[union_df['ENTITY SHORT NAME'] == VOE_bank]

print(VOE_df['NET INCOME (LOSS): BHBC4340'])

43    -66310.0
767        0.0
Name: NET INCOME (LOSS): BHBC4340, dtype: float64
